In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# xavier initialization function

In [2]:
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval = low, maxval = high, dtype = tf.float32)

# additive gaussian noise autoencoder class

In [3]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus,
                optimizer=tf.train.AdamOptimizer(), scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        
        # define network architecture
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(
                                self.x + scale * tf.random_normal((n_input,)),
                                self.weights['w1']), self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
        
        # loss function
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(
                            self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        
    # initialize weights function
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights
    
    # cost function and partial_fit function
    def partial_fit(self, x):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x: x, self.scale: self.training_scale})
        return cost
    
    # only cost function
    def calc_total_cost(self, x):
        return self.sess.run(self.cost, feed_dict={self.x: x, self.scale: self.training_scale})
    
    # transform function
    def transform(self, x):
        return self.sess.run(self.hidden, feed_dict={self.x: x, self.scale: self.training_scale})
    
    # generate function
    def generate(self, hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights["b1"])
        return self.see.run(self.reconstruction, feed_dict={self.hidden: hidden})
    
    # reconstruct function
    def reconstruct(self, x):
        return self.sess.run(self.reconstruction, feed_dict={self.x: x, self.scale: self.training_scale})
    
    # getWeights function
    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    # getBiases function
    def getBiases(self):
        return self.sess.run(self.weights['b1'])

# standard scale function

In [4]:
def standard_scale(x_train, x_test):
    preprocessor = prep.StandardScaler().fit(x_train)
    x_train = preprocessor.transform(x_train)
    x_test = preprocessor.transform(x_test)
    return x_train, x_test

# random block function

In [5]:
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data)-batch_size)
    return data[start_index: (start_index + batch_size)]

# load data

In [6]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
x_train, x_test = standard_scale(mnist.train.images, mnist.test.images)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


# train

In [7]:
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784, 
                                              n_hidden=200,
                                              transfer_function=tf.nn.softplus,
                                              optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                                              scale=0.01)

for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(x_train, batch_size)
        
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
        
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        
print("Total cost: " + str(autoencoder.calc_total_cost(x_test)))

Epoch: 0001 cost= 19597.993212500
Epoch: 0002 cost= 12704.766622727
Epoch: 0003 cost= 10857.862907386
Epoch: 0004 cost= 9421.311528977
Epoch: 0005 cost= 9975.754668182
Epoch: 0006 cost= 9478.295655114
Epoch: 0007 cost= 9139.745199432
Epoch: 0008 cost= 9377.190157955
Epoch: 0009 cost= 9079.506094886
Epoch: 0010 cost= 8034.588703409
Epoch: 0011 cost= 8930.324689773
Epoch: 0012 cost= 8712.671421591
Epoch: 0013 cost= 8299.617280114
Epoch: 0014 cost= 7552.333994886
Epoch: 0015 cost= 7714.064661932
Epoch: 0016 cost= 7781.850780114
Epoch: 0017 cost= 7859.792118750
Epoch: 0018 cost= 7932.490422159
Epoch: 0019 cost= 8594.928602273
Epoch: 0020 cost= 8295.253238636
Total cost: 648657.0
